<a href="https://colab.research.google.com/github/Nneji123/Super-Resolution-Model/blob/main/Super_Resolution_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone the Repository and install the requirements_colab.txt file

In [ ]:
!git clone https://github.com/Nneji123/Super-Resolution-Model.git

In [ ]:
!cp -R /content/Super-Resolution-Model/requirements_colab.txt /content
!pip install -r requirements_colab.txt

# Copy the contents of the cloned Repository to the working directory

In [3]:
!cp -R /content/Super-Resolution-Model/RRDB_ESRGAN_x4.pth /content
!cp -R /content/Super-Resolution-Model/RRDBNet_arch.py /content

# Download and install Ngrok which is used for exposing the localhost so we can view the API with Google Colab.

In [ ]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz

In [ ]:
!tar -xvzf /content/ngrok-v3-stable-linux-amd64.tgz

In [ ]:
!ngrok authtoken 29vr8YhWZ9CDHrUq2kr0CpUA0e8_6ik4hU5GjJZeAYagTH5i4

# Instantiate the Colab Code module and the run the next cell to create the fastapi app.

In [ ]:
from colabcode import ColabCode


In [ ]:
import uvicorn
from fastapi import FastAPI, File, UploadFile, Response
from fastapi.responses import StreamingResponse, FileResponse
import numpy as np
import io
from PIL import Image
import torch
import RRDBNet_arch as arch 
import cv2

app = FastAPI()

model_path = 'RRDB_ESRGAN_x4.pth'  # models/RRDB_ESRGAN_x4.pth OR models/RRDB_PSNR_x4.pth
#device = torch.device('cuda')  # if you want to run on CPU, change 'cuda' -> cpu
device = torch.device('cpu')

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

@app.post("/")
async def root(file: UploadFile = File(...)):
    contents = io.BytesIO(await file.read())
    file_bytes = np.asarray(bytearray(contents.read()), dtype=np.uint8)
    img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    

    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()

    
    #cv2.imwrite('results.png', output)
    #return Response('results.png', media_type="image/png")
    res, im_png = cv2.imencode(".png", output)
    return StreamingResponse(io.BytesIO(im_png.tobytes()), media_type="image/png")

In [ ]:
cc = ColabCode(port=12000, code=False)

# The API can then be viewed using the Public Url Generated by Colab Code.

In [ ]:
cc.run_app(app=app)

INFO:     Started server process [486]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://3a17-34-80-240-209.ngrok.io" -> "http://localhost:12000"
INFO:     102.89.39.104:0 - "GET / HTTP/1.1" 405 Method Not Allowed
INFO:     102.89.39.104:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     102.89.39.104:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     102.89.39.104:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     102.89.39.104:0 - "POST / HTTP/1.1" 200 OK
INFO:     102.89.38.124:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     102.89.38.124:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     102.89.38.124:0 - "POST / HTTP/1.1" 200 OK
INFO:     102.89.47.253:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     197.149.127.196:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     197.149.127.196:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     102.89.47.253:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     102.89.38.124:0 - "POST / HTTP/1.1" 200 OK
INFO:     102.89.38.124:0 - "POST / HTTP/1.1" 200 OK
INFO:     102.89.38.124:0 - "POST / HTTP/1.1" 200 OK
INFO:     102.89.38.16:0 - "PO